In [2]:
import os
import cv2
import mediapipe as mp
import pandas as pd

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)
kategori = ['tendangan lurus', 'tendangan sabit']  # pastikan nama folder sesuai
data_all = []

for label in kategori:
    folder = f'dataset/tendangan/{label}'
    for filename in os.listdir(folder):
        if filename.endswith('.mp4'):
            cap = cv2.VideoCapture(os.path.join(folder, filename))
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(image)
                if results.pose_landmarks:
                    row = []
                    for lm in results.pose_landmarks.landmark:
                        row.extend([lm.x, lm.y, lm.z, lm.visibility])
                    row.append(label)
                    data_all.append(row)
            cap.release()

pose.close()

# Simpan ke CSV
num_landmarks = 33 * 4  # x, y, z, v
columns = [f'{i}_{axis}' for i in range(33) for axis in ['x', 'y', 'z', 'v']]
columns.append('label')

df = pd.DataFrame(data_all, columns=columns)
df.to_csv('csv/pose_dataset_tendangan2.csv', index=False)
print("✅ Dataset disimpan ke pose_dataset_tendangan2.csv")


✅ Dataset disimpan ke pose_dataset_tendangan2.csv


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# ✅ 1. Baca dataset
df = pd.read_csv('csv/pose_dataset_tendangan2.csv')

# ✅ 2. Ambil hanya titik-titik kaki (6 keypoint × 4 = 24 fitur)
selected_indices = [23, 24, 25, 26, 27, 28]
selected_columns = []

for i in selected_indices:
    for axis in ['x', 'y', 'z', 'v']:
        selected_columns.append(f"{i}_{axis}")

X = df[selected_columns].values.astype('float32')
y = LabelEncoder().fit_transform(df['label'])
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# ✅ 3. Buat dan latih model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(y.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[early_stop])

model.save("model/model_tendangan2.h5")
print("✅ Model disimpan ke model/model_tendangan2.h5")


Epoch 1/50


c:\Users\filam\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


367/367 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5305 - loss: 0.6938 - val_accuracy: 0.6736 - val_loss: 0.6460
Epoch 2/50
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6125 - loss: 0.6523 - val_accuracy: 0.7384 - val_loss: 0.5849
Epoch 3/50
367/367 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6925 - loss: 0.5913 - val_accuracy: 0.7868 - val_loss: 0.4940
Epoch 4/50
367/367 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7559 - loss: 0.5174 - val_accuracy: 0.8114 - val_loss: 0.4436
Epoch 5/50
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7858 - loss: 0.4732 - val_accuracy: 0.8663 - val_loss: 0.3755
Epoch 6/50
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8087 - loss: 0.4318 - val_accuracy: 0.8629 - val_loss: 0.3461
Epoch 7/50
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8333 - loss: 0.3883 - val_accuracy: 0.8336 - val_loss: 0.3499
Epoch 8/50
367/367 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8505 - loss: 0.3601 - val_accuracy: 0.8806 - val_

✅ Model disimpan ke model/model_tendangan2.h5


In [4]:
import tensorflow as tf
from tensorflow import keras

# Load model dari .h5
model = keras.models.load_model("model/model_tendangan2.h5")

# Konversi ke TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan ke file
with open("model/model_tendangan2.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Model TFLite disimpan ke model_pukulan2.tflite")


INFO:tensorflow:Assets written to: C:\Users\filam\AppData\Local\Temp\tmpn6za2oa9\assets


INFO:tensorflow:Assets written to: C:\Users\filam\AppData\Local\Temp\tmpn6za2oa9\assets


Saved artifact at 'C:\Users\filam\AppData\Local\Temp\tmpn6za2oa9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 24), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  2092955161552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2092962052240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2092957453392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2092957450320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2092957453968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2092957452624: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Model TFLite disimpan ke model_pukulan2.tflite
